# Analyse  Src

 work with Weakly_2024_07
- use jupyter kernel LSST


- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2024/02/25
- update : 2024/02/29


``Quantum Graphs``: https://tigress-web.princeton.edu/~lkelvin/pipelines/ 

``CalibrateTask``: 
https://github.com/lsst/pipe_tasks/blob/main/python/lsst/pipe/tasks/calibrate.py#L392-L399

 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm


import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch
from astropy.io import fits
import astropy.coordinates as coord
import astropy.units as u
import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

In [ ]:
#df_sel["psfMag"] =  df_sel['psfFlux'].map(convert_fluxtomag)
#df_sel["gaussianMag"] =  df_sel['gaussianFlux'].map(lambda x:-2.50 * np.log10(x) + 31.4)
#df_sel["gaussianMag"] =  df_sel['gaussianFlux'].map(convert_fluxtomag)

def convert_fluxtomag(x) :
    """
    The object and source catalogs store only fluxes. There are hundreds of flux-related columns, 
    and to store them also as magnitudes would be redundant, and a waste of space.
    All flux units are nanojanskys. The AB Magnitudes Wikipedia page provides a concise resource 
    for users unfamiliar with AB magnitudes and jansky fluxes. To convert to AB magnitudes use:
    As demonstrated in Section 2.3.2, to add columns of magnitudes after retrieving columns of flux, users can do this:
    results_table['r_calibMag'] = -2.50 * numpy.log10(results_table['r_calibFlux']) + 31.4
    results_table['r_cModelMag'] = -2.50 * numpy.log10(results_table['r_cModelFlux']) + 31.4
    (from DP0 tutorial)
    """
    return -2.50 * np.log10(x) + 31.4

## Configuration

In [ ]:
#filesource_in="preSourceTable_202312.csv"
filesource_in1="../data/202402/icSrc_202402.csv"
filesource_in2="../data/202402/src_202402.csv"
filevisit_in3 = "../data/202402/ccdVisitTable_202402.csv"
fileisolated_star_sources_in4 = "../data/202402/isolated_star_sources_202402.csv"

## Read files

In addition compute nightobs and seq

In [ ]:
#icSrc
df1 = pd.read_csv(filesource_in1,index_col=0)
df1.dropna(axis=1, how='all',inplace =True)
df1["nightObs"] =df1.apply(lambda x: x['visitId']//100_000, axis=1)
df1["seq"] = df1["visitId"] - df1["nightObs"]*100_000
df1["slot_ApFlux_instMag"] =  df1["slot_ApFlux_instFlux"].map(convert_fluxtomag)
df1["slot_PsfFlux_instMag"] =  df1["slot_PsfFlux_instFlux"].map(convert_fluxtomag)

#df1["base_ApFlux_instMag"] =  df1["base_ApFlux_instFlux"].map(convert_fluxtomag)
df1["base_PsfFlux_instMag"] =  df1["base_PsfFlux_instFlux"].map(convert_fluxtomag)

In [ ]:
list(df1.columns)

In [ ]:
#src
df2 = pd.read_csv(filesource_in2,index_col=0)
df2.dropna(axis=1, how='all',inplace =True)
df2["nightObs"] =df2.apply(lambda x: x['visitId']//100_000, axis=1)
df2["seq"] = df2["visitId"] - df2["nightObs"]*100_000

In [ ]:
#ccd visits
df3 = pd.read_csv(filevisit_in3,index_col=0)
df3.dropna(axis=1, how='all',inplace =True)
df3["nightObs"] =df3.apply(lambda x: x['visitId']//100_000, axis=1)
df3["seq"] = df3["visitId"] - df3["nightObs"]*100_000

In [ ]:
# isolated sources
df4 = pd.read_csv(fileisolated_star_sources_in4,index_col=0)
df4.dropna(axis=1, how='all',inplace =True)
df4 = df4.rename(columns={"visit": "visitId"})
df4["nightObs"] =df4.apply(lambda x: x['visitId']//100_000, axis=1)
df4["seq"] = df4["visitId"] - df4["nightObs"]*100_000

In [ ]:
col_df1 = df1.columns.tolist()
col_df2 = df2.columns.tolist()
col_df3 = df3.columns.tolist()
col_df4 = df4.columns.tolist()

## Dump tables

In [ ]:
#icSrc
df1.sort_values(by=["visitId","id"], inplace=True) 
df1

In [ ]:
np.sort(df1["nightObs"].unique())

In [ ]:
column_names = ["slot_ApFlux_instFlux","slot_PsfFlux_instFlux","base_PsfFlux_instFlux"]
fig,ax = plt.subplots(figsize=(16,8))
#axes = df1.hist(column= column_names,by="nightObs",bins=20,ax=ax,xrot=0,range=(16,23),legend=True)
axes = df1.hist(column= column_names,histtype='bar',bins=50,ax=ax,xrot=0,range=(-1000,110000),legend=True)
plt.suptitle(column_names)
plt.tight_layout()

In [ ]:
column_names = ["slot_ApFlux_instMag","slot_PsfFlux_instMag","base_PsfFlux_instMag"]
fig,ax = plt.subplots(figsize=(16,8))
#axes = df1.hist(column= column_names,by="nightObs",bins=20,ax=ax,xrot=0,range=(16,23),legend=True)
axes = df1.hist(column= column_names,bins=50,ax=ax,xrot=0,range=(0,30),legend=True)
plt.suptitle(column_names)

In [ ]:
#Src
df2.sort_values(by=["visitId","id"], inplace=True) 
df2

In [ ]:
np.sort(df2["nightObs"].unique())

In [ ]:
#CCd visit
df3.sort_values(by=["visitId"], inplace=True) 
df3

In [ ]:
np.sort(df3["nightObs"].unique())

In [ ]:
# isolated source
df4.sort_values(by=["visitId"], inplace=True) 
df4

In [ ]:
np.sort(df4["nightObs"].unique())

## Select Date

In [ ]:
DATEOBS =  20240130

In [ ]:
cut1 = df1["nightObs"] == DATEOBS
cut2 = df2["nightObs"] == DATEOBS
cut3 = df3["nightObs"] == DATEOBS
cut4 = df4["nightObs"] == DATEOBS

In [ ]:
df1 = df1[cut1]
df2 = df2[cut2]
df3 = df3[cut3]
df4 = df4[cut4]

In [ ]:
df4[['visitId','seq','tract']]

## Merge

In [ ]:
df1_m = df1.merge(df3,how='left',left_on="visitId",right_on="visitId")

In [ ]:
df1_m

In [ ]:
df2_m = df2.merge(df3,how='left',left_on="visitId",right_on="visitId")

In [ ]:
df2_m 

In [ ]:
df_sel.plot.scatter('x','y',c='ccdVisitId',colormap='viridis')

In [ ]:
df_sel.plot.scatter('ra','dec',c='ccdVisitId',colormap='hsv',grid=True,figsize=(10,5))

In [ ]:
df_sel.plot.scatter('coord_ra','coord_dec',c='ccdVisitId',colormap='hsv',grid=True,figsize=(10,5))

In [ ]:
df_sel['coord_ra'].plot(kind="hist",bins=50 ,range=(0,360),logy=True,title="coord_ra",grid=True,figsize=(10,5))

In [ ]:
df_sel['coord_dec'].plot(kind="hist",bins=50 ,range=(-90,10),logy=True,title="coord_dec",grid=True,figsize=(10,5))

In [ ]:
cut_skyloc = ((df_sel['dec'] < -30) & (df_sel['dec'] > -40) & (df_sel['ra'] > 100) & (df_sel['ra'] < 150))

In [ ]:
df_sel = df_sel[cut_skyloc]

In [ ]:
df_sel['psfMag'].plot(kind="hist",bins=50,range=(5,30),logy=True,grid=True,title="psMag")

In [ ]:
df_sel['gaussianMag'].plot(kind="hist",bins=50 ,range=(5,30),logy=True,grid=True,title="gaussianMag")

In [ ]:
df_sel

In [ ]:
#df_sel.groupby(df_sel['dateobs'].dt.date).size().plot()
datetime = pd.to_datetime(df_sel['dateobs'])

In [ ]:
df_sel['dateobs'].max()

In [ ]:
df_sel["date"] = df["dateobs"].astype("datetime64")

In [ ]:
df_sel

In [ ]:
df_time = df_sel[["date"]]

In [ ]:
df_time

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax = (df_time["date"].groupby(df_time["date"].dt.minute)
                         .count()).plot(kind="bar")